<a href="https://colab.research.google.com/github/Circle-vip/Yandex_practicum/blob/main/41794fc9_3089_4e9e_81c2_a098bdfbc07a_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Исследование рынка общественного питания в Москве.

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Описание проекта:

 - Исследовать соотношение видов объектов общественного питания по количеству.
 - Исследовать соотношение сетевых и несетевых заведений по количеству. 
 - Выяснить, для какого вида объекта общественного питания характерно сетевое распространение?
 - Выяснить, что характерно для сетевых заведений: много заведений с небольшим числом посадочных мест в каждом или мало заведений с большим количеством посадочных мест?
 - Для каждого вида объекта общественного питания описать среднее количество посадочных мест. Какой вид предоставляет в среднем самое большое количество посадочных мест?
 - Выделить в отдельный столбец информацию об улице из столбца address .

 А также визуализировать результаты и подготовить презентацию.

***Цель:*** Найти место для ресторана с роботами-официантами. Порекомендовать кол-во посадочных мест и вида заведения.


**Данные:**

/content/drive/MyDrive/Яндекс Практикум/Проект №7, общепиты, plotly/rest_data (1).csv

Таблица rest_data:

- id — идентификатор объекта;
- object_name — название объекта общественного питания;
- chain — сетевой ресторан;
- object_type — тип объекта общественного питания;
- address — адрес;
- number — количество посадочных мест.








## Обзор данных

In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from plotly import graph_objects as go
from scipy import stats
from numpy import median
import warnings
from io import BytesIO
import requests
import plotly.io as pio

In [11]:
#Устанавливаем единую тему для всех графиков
pio.templates.default = "plotly_white"

px.defaults.template = "plotly_white"
px.defaults.color_continuous_scale = px.colors.sequential.Agsunset
px.defaults.width = 900
px.defaults.height = 550

In [12]:
#Чтобы окна об ошибках не мешали
warnings.filterwarnings('ignore')

In [13]:
df = pd.read_csv('/content/drive/MyDrive/Яндекс Практикум/Проект №7, общепиты, plotly/rest_data (1).csv')

In [14]:
# Фуникция первичного знакомства с данными
# Функция, которая сразу выводит всю информацию, принимая на вход название датасета
def data_view(data):
    #Сразу приводим названия всех колонок к змеиному регистру.    
    data.columns = data.columns. \
                                str.replace(r'([a-z0-9])([A-Z])' , r'\1_\2', regex = True). \
                                str.replace(r'(.)([A-Z][a-z]+)', r'\1_\2', regex = True). \
                                str.replace(r'([a-z0-9])([A-Z])', r'\1_\2', regex = True). \
                                str.replace(r'__([A-Z])', r'_\1', regex = True). \
                                str.replace(r'__([a-z0-9])', r'_\1', regex = True)
    data.columns = data.columns.str.lower()
    print()
    print('------------------ Первые 5 строк данных -------------------')
    display(data.head()) # Выводим первые 5 строк
    print()
    print('------------------ Последние 5 строк данных -----------------')
    display(data.tail()) # Выводим последние 5 строк
    print()
    print('------------------ Информация по датасету --------------------')
    print()
    display(data.info()) # Выводим инфо по датасету
    print()
    # Уточняем количество пропусков по столбцам. Берем сумму пропусков и процентную долю пропусков
    print('------------------ Пропуски и уникальные значения (без учета пропусков) ---------------' )
    display(pd.concat([data.isna().sum().rename('nan_count'), 
                       data.isna().mean().rename('nan_percent'),
                       data.nunique().rename('unique_values')], 
                      axis=1).style.format({'nan_percent' : '{:.2%}'})) 
    print()    
    print('------------------ Дубликаты -----------------------')
    print()
    print('Количество явных дубликатов:', data.duplicated().sum())
    print()

In [15]:
data_view(df)


------------------ Первые 5 строк данных -------------------


,id,object_name,chain,object_type,address,number
0,151635,СМЕТАНА,нет,кафе,"город Москва, улица Егора Абакумова, дом 9",48
1,77874,Родник,нет,кафе,"город Москва, улица Талалихина, дом 2/1, корпус 1",35
2,24309,Кафе «Академия»,нет,кафе,"город Москва, Абельмановская улица, дом 6",95
3,21894,ПИЦЦЕТОРИЯ,да,кафе,"город Москва, Абрамцевская улица, дом 1",40
4,119365,Кафе «Вишневая метель»,нет,кафе,"город Москва, Абрамцевская улица, дом 9, корпус 1",50



------------------ Последние 5 строк данных -----------------


,id,object_name,chain,object_type,address,number
15361,208537,Шоколадница,да,кафе,"город Москва, 3-й Крутицкий переулок, дом 18",50
15362,209264,Шоколадница,да,кафе,"город Москва, улица Земляной Вал, дом 33",10
15363,209186,Шоколадница,да,кафе,"город Москва, улица Земляной Вал, дом 33",20
15364,221900,Шоколадница,да,кафе,"город Москва, поселение Московский, Киевское ш...",36
15365,222535,Шоколадница,да,кафе,"город Москва, Ходынский бульвар, дом 4",10



------------------ Информация по датасету --------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15366 entries, 0 to 15365
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           15366 non-null  int64 
 1   object_name  15366 non-null  object
 2   chain        15366 non-null  object
 3   object_type  15366 non-null  object
 4   address      15366 non-null  object
 5   number       15366 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 720.4+ KB


None


------------------ Пропуски и уникальные значения (без учета пропусков) ---------------


,nan_count,nan_percent,unique_values
id,0,0.00%,15366
object_name,0,0.00%,10393
chain,0,0.00%,2
object_type,0,0.00%,9
address,0,0.00%,9108
number,0,0.00%,315



------------------ Дубликаты -----------------------

Количество явных дубликатов: 0



При первом взляде на данные дубликаты и пропуски не найдены. 

### Предобработка данных

In [16]:
#Приводим столбцы к нижнему регистру
df['object_name'] = df['object_name'].str.lower()
df['address'] = df['address'].str.lower()
df.head(3)

,id,object_name,chain,object_type,address,number
0,151635,сметана,нет,кафе,"город москва, улица егора абакумова, дом 9",48
1,77874,родник,нет,кафе,"город москва, улица талалихина, дом 2/1, корпус 1",35
2,24309,кафе «академия»,нет,кафе,"город москва, абельмановская улица, дом 6",95


In [17]:
df.duplicated().sum()  #проверила на дубликаты после приведения к нижнему регистру

0

In [18]:
df['chain'].unique()

array(['нет', 'да'], dtype=object)

In [19]:
#Функция для приведения значений  к bool
def to_logical_values(x):
    if x=='да':
        return True
    elif x=='нет':
        return False
df['chain'] = df['chain'].apply(to_logical_values)

df.info()
print('\nТипы данных выгружены корректно, пропусков не обнаружено, в столбце chain заменены значения на логические')
df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15366 entries, 0 to 15365
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           15366 non-null  int64 
 1   object_name  15366 non-null  object
 2   chain        15366 non-null  bool  
 3   object_type  15366 non-null  object
 4   address      15366 non-null  object
 5   number       15366 non-null  int64 
dtypes: bool(1), int64(2), object(3)
memory usage: 615.4+ KB

Типы данных выгружены корректно, пропусков не обнаружено, в столбце chain заменены значения на логические


,id,object_name,chain,object_type,address,number
0,151635,сметана,False,кафе,"город москва, улица егора абакумова, дом 9",48
1,77874,родник,False,кафе,"город москва, улица талалихина, дом 2/1, корпус 1",35
2,24309,кафе «академия»,False,кафе,"город москва, абельмановская улица, дом 6",95
3,21894,пиццетория,True,кафе,"город москва, абрамцевская улица, дом 1",40
4,119365,кафе «вишневая метель»,False,кафе,"город москва, абрамцевская улица, дом 9, корпус 1",50


In [20]:
df['object_type'].unique() #посмотрела типы заведений

array(['кафе', 'столовая', 'закусочная',
       'предприятие быстрого обслуживания', 'ресторан', 'кафетерий',
       'буфет', 'бар', 'магазин (отдел кулинарии)'], dtype=object)

In [21]:
#Поменяла типы на более понятные и короткие
df['object_type'] = df['object_type'].replace('предприятие быстрого обслуживания', 'фаст-фуд')
df['object_type'] = df['object_type'].replace('магазин (отдел кулинарии)', 'кулинария в магазине')

In [22]:
street_types = ['улица', 'аллея', 'бульвар', 'линия', 'набережная', 
                'переулок', 'просека', 'проспект', 'тупик', 'шоссе',
               'площадь', 'проезд']
# определим функцию street_names() которая извлекает названия улиц из адресов
def street_names(x, street_types):
    # исходный адрес приведем к нижнему регистру и разделим в список по разделителю ','
    address_list = x.lower().split(',')
    # если в элементе списка встречается значение из street_types то этот элемент считаем названием улицы
    # иначе названием улицы будет 2 элемент из списка address_list
    for type_street in street_types:
        for part in address_list:
            if type_street in part:
                return part
    return address_list[1]
# в датафрейм correct_data в столбец street запишем названия улиц, применив функцию street_names(топ-10 улиц)
df['street'] = df['address'].apply(lambda x: street_names(x, street_types)).str.strip()

df.head(3)

,id,object_name,chain,object_type,address,number,street
0,151635,сметана,False,кафе,"город москва, улица егора абакумова, дом 9",48,улица егора абакумова
1,77874,родник,False,кафе,"город москва, улица талалихина, дом 2/1, корпус 1",35,улица талалихина
2,24309,кафе «академия»,False,кафе,"город москва, абельмановская улица, дом 6",95,абельмановская улица


In [23]:
#вставила файл с данными улиц и  районов москвы
spreadsheet_id_2 = '1SGsPtXKrEsdoeZcw0VZ8DtR8i8bZCIYxuBSk7ErL7Zs'
file_name_2 = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(spreadsheet_id_2)
r_2 = requests.get(file_name_2)
streets = pd.read_csv(BytesIO(r_2.content))

#привела значения к нижнему регистру
streets['streetname'] = streets['streetname'].str.lower()
streets['area'] = streets['area'].str.lower()

#переименовала колонки
streets.rename(columns = {'streetname':'street', 'area':'district'}, inplace=True)
streets.head()

,street,areaid,okrug,district
0,выставочный переулок,17,ЦАО,пресненский район
1,улица гашека,17,ЦАО,пресненский район
2,большая никитская улица,17,ЦАО,пресненский район
3,глубокий переулок,17,ЦАО,пресненский район
4,большой гнездниковский переулок,17,ЦАО,пресненский район


**Вывод по предобработке:** Дубликатов и пропусков нет. Данные приведены к нужным типам и названиям. После размышлений по поводу неявных дубликатов в названиях заведений, поняла, что лучше их не править, потому что заведения, например по франшизе, могут иметь одно название, но разных хозяев. Или одного хозяина, но разные названия  - макдональдс и макдональдс на выставочной.

## Анализ данных

### Cоотношение видов объектов общественного питания по количеству. 

In [24]:
object_types = (
    df.groupby('object_type', as_index=False)
    .agg({'object_name':'count'})
    .sort_values(by='object_name', ascending=False)
)
#Добавила столбец с соотношением в процентах
object_types['percent'] = (object_types['object_name']/object_types['object_name'].sum()*100).round(1).astype('str') + '%'
object_types

,object_type,object_name,percent
3,кафе,6099,39.7%
7,столовая,2587,16.8%
6,ресторан,2285,14.9%
8,фаст-фуд,1923,12.5%
0,бар,856,5.6%
1,буфет,585,3.8%
4,кафетерий,398,2.6%
2,закусочная,360,2.3%
5,кулинария в магазине,273,1.8%


In [25]:
fig = px.bar(
    object_types,
    x='object_type',
    y='object_name',
    text = 'percent',
    labels = ({'object_type':'Тип объекта'}),
    color='object_type'
)
#добавляем подписи
fig.update_layout(
    title='Соотношение видов заведений общепита по количеству',
    xaxis_title="",
    yaxis_title="Количество")

#Вынгсим значения из столбцов
fig.update_xaxes(tickangle=45)
fig.update_traces(
    textposition = 'outside'
)

fig.show()

Итак, лидирует, с большим отрывом, кафе. 

### Соотношение сетевых и несетевых заведений по количеству.

In [26]:
chain= (
    df.groupby(['object_type', 'chain'],as_index=False)
    .agg({'id':'count'})
    .sort_values(by='id', ascending=False)
)
chain.rename(columns = {'id':'count'}, inplace=True)
chain

,object_type,chain,count
6,кафе,False,4703
14,столовая,False,2584
12,ресторан,False,1741
7,кафе,True,1396
16,фаст-фуд,False,1132
0,бар,False,819
17,фаст-фуд,True,791
2,буфет,False,574
13,ресторан,True,544
8,кафетерий,False,346


In [27]:
fig = go.Figure(
    data=go.Pie(labels=['Несетевые', 'Сетевые'], values=df['chain'].value_counts())
)
fig.update_layout(
    legend_orientation="h",
    legend=dict(x=.5, xanchor="center"),
    title={
        'text':'Соотношение сетевых и несетевых заведений по количеству',
        'y':0.9,
        'x':0.48},
)

Как видно из графика - сетевых заведений всего 19% от общего количества, остальные - несетевые.

### Для какого вида объекта общественного питания характерно сетевое распространение?

In [28]:
chain_types = (
    df[df['chain']==True].groupby('object_type',as_index=False)
    .agg({'object_name':'count'})
    .sort_values(by='object_name', ascending=False)
)
chain_types = chain_types.merge(
    object_types[['object_type', 'object_name']],
    left_on='object_type',
    right_on='object_type'
)
#Переименовала столбцы
chain_types.columns=['object_type', 'chain_objects', 'sum_objects']

#Создала столбец для соотношения
chain_types['percent'] = (chain_types['chain_objects'] / chain_types['chain_objects'].sum()*100).round(1).astype('str')+'%'

#Столбец для отношения к типу заведения
chain_types['percent_of_type'] = (chain_types['chain_objects'] / chain_types['sum_objects']*100).round(2)

#Столбец для графика
chain_types['percent_of_type_text'] = chain_types['percent_of_type'].astype('str')+'%'

chain_types

,object_type,chain_objects,sum_objects,percent,percent_of_type,percent_of_type_text
0,кафе,1396,6099,47.0%,22.89,22.89%
1,фаст-фуд,791,1923,26.7%,41.13,41.13%
2,ресторан,544,2285,18.3%,23.81,23.81%
3,кулинария в магазине,78,273,2.6%,28.57,28.57%
4,закусочная,56,360,1.9%,15.56,15.56%
5,кафетерий,52,398,1.8%,13.07,13.07%
6,бар,37,856,1.2%,4.32,4.32%
7,буфет,11,585,0.4%,1.88,1.88%
8,столовая,3,2587,0.1%,0.12,0.12%


In [29]:
fig = px.bar(
    chain_types,
    x='object_type',
    y='chain_objects',
    color='object_type',
    labels = ({'object_type':'Тип объекта'}),
    text = 'chain_objects'
)
fig.update_xaxes(tickangle=45)

fig.update_layout(
    title='Соотношение сетевых и несетевых по типам заведений',
    xaxis_title="",
    yaxis_title="Количество")


fig.update_traces(
    textposition = 'outside'
)

fig.show()


In [30]:
chain_tp = (
    df.groupby(['object_type', 'chain'],as_index=False)
    .agg({'object_name':'count'})
    .sort_values(by='object_name', ascending=False)
)

chain_tp

,object_type,chain,object_name
6,кафе,False,4703
14,столовая,False,2584
12,ресторан,False,1741
7,кафе,True,1396
16,фаст-фуд,False,1132
0,бар,False,819
17,фаст-фуд,True,791
2,буфет,False,574
13,ресторан,True,544
8,кафетерий,False,346


In [31]:
fig = px.bar(
    chain_tp,
    x='object_type',
    y='object_name',
    color='chain',
    labels = ({'chain':'Сетевые и несетевые'}),
)
fig.update_xaxes(tickangle=30)
fig.update_layout(
    title='Доля сетевых объектов по типам заведений общепита',
    xaxis_title="",
    yaxis_title="Кол-во заведений",
    
)
fig.update_traces(textposition='outside')

In [32]:
#строим график, чтобы было нагляднее
fig = px.bar(
    chain_types.sort_values(by='percent_of_type', ascending = False),
    x='object_type',
    y='percent_of_type',
    color='object_type',
    labels = ({'object_type':'Тип объекта'}),
    text='percent_of_type_text'
)
fig.update_xaxes(tickangle=30)
fig.update_layout(
    title='Доля сетевых объектов по типам заведений общепита',
    xaxis_title="",
    yaxis_title="% заведений"
)
fig.update_traces(textposition='outside')

Итак, больше всего сетевых заведений у таких типов, как - кафе, фаст-фуд, ресторан. Однако, в разрезе самих типов заведений, у фастфуда более 40% от всех заведений  - сетевые, то есть почти половина. Затем, видно, что среди кулинарий развиваться по сети тоже распространено. 

### Что характерно для сетевых заведений: много заведений с небольшим числом посадочных мест в каждом или мало заведений с большим количеством посадочных мест?

In [33]:
#Отделяем сетевые заведения
chain_df = df.query('chain == True')

chain_df = (
    chain_df.groupby('object_name', as_index=False)
    .agg({'id': 'count', 'number': 'median'})
    .sort_values(by='id', ascending=False)    
)

chain_df.columns = ['object_name','objects', 'seat_median']

#Узнать как обстоят дела у самых топовых сетевых заведений
chain_top10 = chain_df.head(10)
display(chain_top10)
print('Среднее кол-во объектов:', chain_top10['objects'].mean())
print('Среднее кол-во посадочных мест:', chain_top10['seat_median'].mean())

,object_name,objects,seat_median
563,шоколадница,157,50.0
25,kfc,155,45.0
330,макдоналдс,151,76.0
109,бургер кинг,137,45.0
521,теремок,94,23.0
311,крошка картошка,90,15.0
159,домино'с пицца,90,16.0
339,милти,72,0.0
505,суши wok,72,6.0
367,папа джонс,51,20.0


Среднее кол-во объектов: 106.9
Среднее кол-во посадочных мест: 29.6


In [34]:
fig = px.scatter(
    chain_df.head(100),
    y='seat_median',
    x='objects',
    color='object_name',
    size='objects',
    size_max=50,
    labels={'seat_median': 'среднее число посадочных мест', 'objects': 'количество заведений в сети'},
    template='plotly_white',
    title='Распределение посадочных мест для сетевых заведений'
)
fig.update_layout(showlegend=False)

fig.show()

fig = px.scatter(
    chain_df.head(10),
    y='seat_median',
    x='objects',
    color='object_name',
    size='objects',
    size_max=60,
    hover_name="object_name", 
    text='seat_median', 
    opacity=.7,
    labels={'seat_median': 'среднее число посадочных мест', 'objects': 'количество заведений в сети', 'object_name':'Название сети:'},
    template='plotly_white',
    title='Распределение посадочных мест в топ-10 сетях'
)


fig.show()

In [35]:
#Строим еще один scatter для наглядности
median_90 = np.percentile(chain_df['seat_median'], 90)
object_90 = np.percentile(chain_df['objects'], 90)
fig = px.scatter(chain_df, y="objects", x="seat_median",color="seat_median", 
                 labels={
                     "seat_median": "Медианное кол-во посадочных мест",
                     "objects": "Кол-во объектов в сети"
                 },
                 render_mode='markers')

fig.add_vline(x=median_90, line_width=3, line_dash="dash", line_color="green")
fig.add_hline(y=object_90, line_width=3, line_dash="dash", line_color="green")
fig.show()
print('90% сетевых заведений имеют не более', median_90, 'посадочных мест')
print('90% сетей имеют не более', object_90.round(0), 'объектов')


90% сетевых заведений имеют не более 125.0 посадочных мест
90% сетей имеют не более 8.0 объектов


**Вывод:** Топовые сетевые заведения имеют в среднем 30-50 посадочных мест и более 90 заведений в сети. Отсюда делаем вывод, что для сетевых заведений характерно много заведений с небольшим числом посадочных мест. 

### Для каждого вида объекта общественного питания опишите среднее количество посадочных мест. Какой вид предоставляет в среднем самое большое количество посадочных мест? 

In [36]:
object_types_seats = (
    df.groupby('object_type',as_index=False)
    .agg({'number': 'median'})
    .round(0)
    .sort_values(by='number', ascending=False)
)   
object_types_seats.columns = ['object_type','seat_median']

object_types_seats

,object_type,seat_median
7,столовая,103.0
6,ресторан,80.0
0,бар,35.0
1,буфет,32.0
3,кафе,30.0
4,кафетерий,6.0
8,фаст-фуд,5.0
2,закусочная,0.0
5,кулинария в магазине,0.0


In [37]:
fig = px.bar(
    object_types_seats,
    x='object_type',
    y='seat_median',
    color='seat_median', 
    text='seat_median',
    labels = ({'seat_median':'Посадочные места'})
)
fig.update_xaxes(tickangle=30)
fig.update_layout(
    title='Среднее количество посадочных мест по типам заведений',
    xaxis_title="",
    yaxis_title="Число посадочных мест"
)
fig.update_traces(textposition='outside')

**Вывод:**  График нам показывает, что посадочных мест больше всего в столовых - в среднем 103 мест. Затем идут рестораны с 80 посад.мест.

### Постройте график топ-10 улиц по количеству объектов общественного питания. В каких районах находятся эти улицы?


In [38]:
# сгруппируем по улице
top_streets = (df
 .groupby('street')
 .agg({'id':'count'})
 .reset_index()
 .sort_values(by = 'id', ascending = False)             
)
#убираем Зеленоград и Сосенское поселение, потому что нас интересует чисто Москва
top_streets['street'] = top_streets['street'].str.strip()
top_streets = top_streets.query('street != "город зеленоград"').reset_index(drop = True)
df = df.query('address != "город зеленоград"')

top_streets = top_streets.query('street != "поселение сосенское"').reset_index(drop = True)
df = df.query('address != "поселение сосенское"')

top_streets

,street,id
0,проспект мира,204
1,профсоюзная улица,183
2,ленинградский проспект,173
3,пресненская набережная,167
4,варшавское шоссе,165
...,...,...
1944,стрельбищенский переулок,1
1945,стрелецкая улица,1
1946,улица клочкова,1
1947,докучаев переулок,1


In [39]:
fig = px.bar(
    top_streets.head(10), 
    y = 'street', 
    x = 'id', 
    color = 'id', 
    text = 'id',
    labels = ({'id':'Число объектов'}),
    orientation='h'
)

# добавим подписи
fig.update_layout(
    xaxis_title = 'Число объектов',
    yaxis_title = '',
    showlegend=False,
    title={
        'text': 'Топ-10 улиц по количеству объектов общественного питания',
        'y':0.94,
        'x':0.56,
        'xanchor': 'center',
        'yanchor': 'top'},
)

# вынесем значения из столбцов
fig.update_traces(
    textposition = 'outside'
)
fig.show()


In [40]:
#Соединяем топ 10 улиц с дф с улицами и районами Москвы
district = top_streets.head(10).merge(streets[['street', 'okrug', 'district']], left_on='street', right_on='street')
district

,street,id,okrug,district
0,проспект мира,204,СВАО,алексеевский район
1,проспект мира,204,СВАО,ярославский район
2,проспект мира,204,СВАО,район марьина роща
3,проспект мира,204,СВАО,останкинский район
4,проспект мира,204,СВАО,район ростокино
5,проспект мира,204,СВАО,район свиблово
6,проспект мира,204,ЦАО,мещанский район
7,профсоюзная улица,183,ЮЗАО,академический район
8,профсоюзная улица,183,ЮЗАО,район черемушки
9,профсоюзная улица,183,ЮЗАО,район ясенево


In [41]:
fig = px.bar(
    district,
    x='street',
    color='district'
)

fig.update_layout(
    xaxis_title = 'Названия улиц',
    yaxis_title = 'Название районов',   
    title={
        'text': 'Топ-10 улиц по районам',
        'y':0.94,
        'x':0.56,
        'xanchor': 'center',
        'yanchor': 'top'},
)
fig.update_xaxes(tickangle=45)
fig.show()


**Вывод:** Лидером по кол-ву заведений общепита является проспект Мира.

проспект Мира находится в:
- Алексеевский район
- Ярославский Район
- Район Марьина роща
- Останкинский район
- Район Ростокино
- Район Свиблово
- Мещанский район

Профсоюзная улица находится в:
- Академический район
- Район Черемушки
- Район Ясенево
- Район Коньково
- Обручевский район
- Район Теплый Стан

Ленинградский проспект находится в:
- Район Аэропорт
- Район Беговой
- Хорошевский район
- Район Сокол

Пресненская набережная находится в:
- Пресненский район

Варшавское шоссе находится в:
- Район Чертаново Центральное
- Район Чертаново Северное
- Район Чертаново Южное
- Донской район
- Район Нагатино-Садовники
- Нагорный район
- Район Северное Бутово
- Район Южное Бутово

Ленинский проспект находится в:
- Район Проспект Вернадского
- Район Тропарево-Никулино
- Донской район
- Район Гагаринский
- Ломоносовский район
- Обручевский район
- Район Теплый Стан
- Район Якиманка

проспект Вернадского находится в:
- Район Проспект Вернадского
- Район Раменки
- Район Тропарево-Никулино
- Район Гагаринский
- Ломоносовский район

Кутузовский проспект находится в:
- Район Дорогомилово
- Район Фили-Давыдково

Каширское шоссе находится в:
- Район Москворечье-Сабурово
- Район Нагатино-Садовники
- Район Орехово-Борисово Южное
- Район Орехово-Борисово Северное

Кировоградская улица находится в:
- Район Чертаново Центральное
- Район Чертаново Северное
- Район Чертаново Южное


<div class="alert alert-info"> <b>Комментарий студента:</b> Я очень много времени убила на то, чтобы выяснить, как сделать график с картой москвы и нанести на нее маркеры, но в итоге не поняла как это правильно сделать. Я нашла как сделать карту, нашла что импортировать и методы, но вот с маркерами беда. У меня получилось это сделать в power bi, но в итоге и тут засада, потому что я не знаю как в юпитер загрузить документ из power bi. В итоге сделала bar, но мне не нравится как он выглядит, если честно. Если у вас есть какая то подсказка по этому поводу или код для примера, то буду очень благодарна. </div>

### Найдите число улиц с одним объектом общественного питания. В каких районах Москвы находятся эти улицы?

In [ ]:
streets_1_object = top_streets[top_streets['id']==1]
print('Количество улиц с 1 объектом общепита: ', len(streets_1_object))

Количество улиц с 1 объектом общепита:  571


In [ ]:
#Соединяем получившуюся таблицу с дф с улицами и районами Москвы
streets_1_object_merged = streets_1_object.merge(
    streets[['street', 'district']],
    left_on='street',
    right_on='street'
)
#Группируем по району и оставляем топ-10 районов
districts_streets_1_object = streets_1_object_merged.groupby('district', as_index=False).agg({'street':'nunique'}).rename(columns={'street':'count_streets'})
districts_streets_1_object.sort_values('count_streets', ascending=False).head(10)

,district,count_streets
95,таганский район,27
81,район хамовники,26
3,басманный район,25
96,тверской район,20
21,пресненский район,20
22,район арбат,18
52,район марьина роща,18
13,мещанский район,16
75,район сокольники,15
34,район замоскворечье,14


In [ ]:
fig = px.bar(
    districts_streets_1_object.sort_values('count_streets', ascending=False).head(10),
    x='district',
    y='count_streets',
    color='count_streets',
    labels = ({'count_streets':'Число улиц'})
)

fig.update_layout(
    xaxis_title = '',
    yaxis_title = 'Число улиц',   
    title={
        'text': 'Топ-10 районов с 1 заведением общепита на улице.',
        'y':0.90,
        'x':0.50,
        'xanchor': 'center',
        'yanchor': 'top'}
)
fig.update_xaxes(tickangle=45)
fig.show()

**Вывод:** Все эти районы находятся в пределах 3-го Транспортного кольца Москвы. 

## Общий вывод:

Целью было найти место для ресторана с роботами-официантами. А так же порекомендовать кол-во посадочных мест и тип заведения.

- По типу заведения в Москве лидирует, с большим отрывом, тип "кафе".
- Несетевых заведений значительно больше - 80% рынка.
- Больше всех по сети развиваются - кафе, рестораны быстрого питания(фаст-фуд) и рестораны.
- В разрезе самих типов заведений у фаст-фуда более 40% от всех заведений именно сетевые.
- Для сетевых заведений характерно небольшое кол-во посадочных мест(30-50) и много самих заведений.
- В среднем, больше всего посадочных мест имеют столовые(более 100 мест), затем идут рестораны(до 100) и буфет(50).
- Улица, с самым большим кол-во заведений общепита - Проспект Мира.
- Так же, было обнаружено 557 улиц имеющих всего 1 заведение. 

По итогам анализа, могу рекомендовать открывать "кафе", не более 100 посадочных мест. На одной из топ-10 улиц по кол-ву объектов общепита, потому что конкуренция полезна для бизнеса и это, очевидно, самые оживленные улицы в городе. Если планируете создавать сеть и развиваться по ней, рекомендую уменьшить кол-во посадочных мест до 30-50 мест.

Презентация: <https://disk.yandex.ru/i/FJl_RDqiY4tfYA>